题目描述：分析题目附件中的代码，根据服务端所给出的counter和token，伪造出一个可以通过验证的counter和token，并提交至服务端  
题目附件中的代码：

In [55]:
import binascii
from gmssl import sm3, func
from Crypto.Util.number import *

In [56]:
## 服务端代码

# 读取HMAC key文件
def read_hmac_key(file_path):
    with open(file_path, 'rb') as f:
        hmac_key = f.read().strip()
    return hmac_key
    
# 生成token
def generate_token(hmac_key, counter):
    # 如果HMAC_KEY长度不足32字节，则在末尾补0，超过64字节则截断
    if len(hmac_key) < 32:
        hmac_key = hmac_key.ljust(32, b'\x00')
    elif len(hmac_key) > 32:
        hmac_key = hmac_key[:32]
    # 将计数器转换为字节表示
    counter_bytes = counter.to_bytes((counter.bit_length() + 7) // 8, 'big')
    # print("counter_bytes:", binascii.hexlify(counter_bytes))
    tobe_hashed = bytearray(hmac_key + counter_bytes)
    # print("tobe_hashed:", binascii.hexlify(tobe_hashed))
    # 使用SM3算法计算哈希值
    sm3_hash = sm3.sm3_hash(tobe_hashed)
    # 将SM3的哈希值转换为十六进制字符串作为token
    token = sm3_hash
    return token

current_counter = 0
def verify_token(hmac_key, counter, token):
    # 生成token
    generated_token = generate_token(hmac_key, counter)
    global current_counter
    # 比较生成的token和输入的token是否相同
    if generated_token == token:
        if counter & 0xFFFFFFFF > current_counter:
            current_counter = counter & 0xFFFFFFFF
            print("current_counter: ", hex(current_counter))
            return "Success"
        else:
            return "Error: counter must be increasing"
    else:
        return "Error: token not match"

# 假设HMAC key文件路径
hmac_key_file = 'hmac_key.txt'
# 读取HMAC key
hmac_key = read_hmac_key(hmac_key_file)

In [57]:
# 假设计数器值
counter = 0x12345678

# 生成token
token = generate_token(hmac_key, counter)
print("Generated token:", token)
print(verify_token(hmac_key, counter, token))

Generated token: e2ba86dfbb76f072f04127e8bfd7204b3a7fc319ac3af02ebc382f468e280a31
current_counter:  0x12345678
Success


现在需要根据上面得出的token和counter，伪造出新的token和counter，并通过验证   
答案

In [58]:
token = int(token, 16)

counter_new = bytes.fromhex(hex(counter)[2:]) 
counter_new += b'\x80' + b'\x00' * 19    # padding
counter_new += b'\x00' * 6 + b'\x01\x20' # len
counter_new += b'\xff' * 4               # counter_append
last_block = b'\xff' * 4
last_block += b'\x80' + b'\x00' * 51     # padding
last_block += b'\x00' * 6 + b'\x02\x20'  # len

prefHashValue = bytes.fromhex(hex(token)[2:])
prefHashValue = [bytes_to_long(prefHashValue[i:i+4]) for i in range(0, 32, 4)]
NewHashValue = sm3.sm3_cf(prefHashValue, func.bytes_to_list(last_block))
NewHashValue = ''.join(['%08x'%val for val in NewHashValue])

print('counter:', counter_new.hex())
print('token:', NewHashValue)

counter: 1234567880000000000000000000000000000000000000000000000000000120ffffffff
token: 9f2b726204878b430995b73b6f6637c4e30e96147e9dfb88988bba003366e768


In [59]:
# 验证
print(verify_token(hmac_key, int.from_bytes(counter_new, byteorder='big'), NewHashValue))

current_counter:  0xffffffff
Success
